In [ ]:
# 📊 Unified Sales & Inventory Analysis  
## Multi-Account MercadoLibre

This project consolidates sales and inventory data from **two different MercadoLibre seller accounts** that sell the **same products**.  
The goal is to create **actionable business reports** focused on inventory restocking, profitability, and sales performance.

---

## 🧠 Business Context

The company operates with two MercadoLibre accounts.

Each account generates its own sales report.  
A third file (`union.csv`) maps equivalent products between both accounts, allowing metrics to be **combined at product level**.

---

## 🎯 Objectives

- Unify sales metrics from two accounts
- Calculate profitability per product
- Identify products that require urgent restocking
- Detect top-selling and high-margin products
- Generate Excel reports ready for business users

---

## ⚙️ Complete Data Pipeline

> The following cell contains the **entire pipeline**, from data loading to Excel report generation.

In [ ]:
import pandas as pd

def sum_columns(df, base_col):
    """
    Sum duplicated columns created after merge (_x, _y)
    and remove the original columns.
    """
    df[base_col] = df[f"{base_col}_x"].fillna(0) + df[f"{base_col}_y"].fillna(0)
    df.drop(columns=[f"{base_col}_x", f"{base_col}_y"], inplace=True)
    return df


def clean_percentage(series):
    """
    Convert percentage strings like '12,5%' to float.
    """
    return (
        series.astype(str)
              .str.replace("%", "", regex=False)
              .str.replace(",", ".", regex=False)
              .astype(float)
              .fillna(0)
    )

# Load data from CSV files
df_pikeke = pd.read_csv("pikeke_reporte.csv")
df_mc = pd.read_csv("solucionesmc_reporte.csv")
df_union = pd.read_csv("union.csv")

# Merge datasets using product mapping
df = (
    df_union
    .merge(df_pikeke, on="id_pikeke", how="left")
    .merge(df_mc, on="id_mc", how="left")
)

# Aggregate main performance metrics
df = sum_columns(df, "Visitas únicas")
df = sum_columns(df, "Cantidad de ventas")
df = sum_columns(df, "Compradores únicos")
df = sum_columns(df, "Unidades vendidas")
df = sum_columns(df, "Ventas brutas (VES)")

# Participation percentage
for col in ["% de participación_x", "% de participación_y"]:
    df[col] = clean_percentage(df[col])

df["participacion"] = df["% de participación_x"] + df["% de participación_y"]
df.drop(columns=["% de participación_x", "% de participación_y"], inplace=True)

# Conversion rate
df["conversion_visitas"] = (
    clean_percentage(df["Conversión de visitas a ventas_x"]) +
    clean_percentage(df["Conversión de visitas a ventas_y"])
)

df.drop(columns=[
    "Conversión de visitas a ventas_x",
    "Conversión de visitas a compradores_x",
    "Conversión de visitas a ventas_y",
    "Conversión de visitas a compradores_y",
    "Estado actual_y",
    "Publicación_y"
], inplace=True)

# Normalize column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df = df.rename(columns={
    "publicación_x": "publicacion",
    "estado_actual_x": "estado",
    "comparedores_unicos": "compradores_unicos"
})

# Profit margin calculation (safe)
df["ganancia_pct"] = (
    (df["venta"] - df["costo"]) / df["costo"]
).replace([float("inf"), -float("inf")], 0) * 100

df = df.round(2)

# Export full dataset
df.sort_values("venta").to_excel("unionxventa.xlsx", index=False)

# Aggregate by product
df_final = df.groupby("publicacion", as_index=False).agg({
    "id_mc": "first",
    "id_pikeke": "first",
    "venta": "first",
    "costo": "first",
    "estado": "first",
    "visitas_unicas": "sum",
    "unidades_vendidas": "sum",
    "ventas_brutas": "sum",
    "participacion": "mean",
    "ganancia_pct": "first"
})

df_final.to_csv("union_agrupada.csv", index=False)
df_final.to_excel("union_agrupada.xlsx", index=False)

# Business reports
df_urgente = df_final[
    (df_final["ganancia_pct"] > 60) &
    (df_final["estado"] == "Inactiva") &
    (df_final["unidades_vendidas"] > 24)
]
df_urgente.to_excel("Pedir_Urgente_Agotados.xlsx", index=False)

df_reponer = df_final[
    (df_final["ganancia_pct"] > 60) &
    (df_final["estado"] == "Activa") &
    (df_final["unidades_vendidas"] > 24)
]
df_reponer.to_excel("Pedir_Activas_Reponer_Stock.xlsx", index=False)

df_top = df_final[
    (df_final["ganancia_pct"] > 60) &
    (df_final["unidades_vendidas"] > 100)
]
df_top.to_excel("Pedir_Top_100_Ventas.xlsx", index=False)
